In [11]:
def left_rotate(value, bits):
    return ((value << bits) | (value >> (32 - bits))) & 0xffffffff

def preprocess(message):
    message = bytearray(message, 'utf-8')
    original_len_in_bits = (8 * len(message)) & 0xffffffffffffffff
    
    message.append(0x80)
    
    while (len(message) * 8) % 512 != 448:
        message.append(0)
    
    message += original_len_in_bits.to_bytes(8, byteorder='big')
    return message

def process_chunk(chunk, h0, h1, h2, h3, h4):
    w = [0] * 80
    for i in range(16):
        w[i] = int.from_bytes(chunk[i*4:i*4+4], byteorder='big')
    
    for i in range(16, 80):
        w[i] = left_rotate(w[i-3] ^ w[i-8] ^ w[i-14] ^ w[i-16], 1)
    
    a = h0
    b = h1
    c = h2
    d = h3
    e = h4
    
    for i in range(80):
        if 0 <= i <= 19:
            f = (b & c) | ((~b) & d)
            k = 0x5A827999
        elif 20 <= i <= 39:
            f = b ^ c ^ d
            k = 0x6ED9EBA1
        elif 40 <= i <= 59:
            f = (b & c) | (b & d) | (c & d)
            k = 0x8F1BBCDC
        elif 60 <= i <= 79:
            f = b ^ c ^ d
            k = 0xCA62C1D6
        
        temp = (left_rotate(a, 5) + f + e + k + w[i]) & 0xffffffff
        e = d
        d = c
        c = left_rotate(b, 30)
        b = a
        a = temp
    
    h0 = (h0 + a) & 0xffffffff
    h1 = (h1 + b) & 0xffffffff
    h2 = (h2 + c) & 0xffffffff
    h3 = (h3 + d) & 0xffffffff
    h4 = (h4 + e) & 0xffffffff
    
    return h0, h1, h2, h3, h4

def sha1(data):
    h0 = 0x67452301
    h1 = 0xEFCDAB89
    h2 = 0x98BADCFE
    h3 = 0x10325476
    h4 = 0xC3D2E1F0
    
    message = preprocess(data)
    
    for i in range(0, len(message), 64):
        h0, h1, h2, h3, h4 = process_chunk(message[i:i+64], h0, h1, h2, h3, h4)
    
    return ''.join(f'{x:08x}' for x in (h0, h1, h2, h3, h4))

def test_sha1():
    test_vectors = {
        "": "da39a3ee5e6b4b0d3255bfef95601890afd80709",
        "abc": "a9993e364706816aba3e25717850c26c9cd0d89d",
        "message digest": "c12252ceda8be8994d5fa0290a47231c1d16aae3",
        "secure hash algorithm": "f30ceb4d2dc9c9ac5cdcd7d1c1c8b6db4e7fadd0",
        "SHA-1 is a cryptographic hash function": "d7b51b5f98e8ed2f5749c25d5f858d473a5b3d9f"
    }
    
    for msg, expected in test_vectors.items():
        actual = sha1(msg)
        if actual != expected:
            print(f"Test failed for: {msg}")
            print(f"Expected: {expected}")
            print(f"Actual:   {actual}")
            print() 
    print("All tests passed!")

user_input = input("Enter a string to hash using SHA-1: ")
hashed_value = sha1(user_input)
print(f"SHA-1 Hash of '{user_input}': {hashed_value}")


Enter a string to hash using SHA-1:  kerala


SHA-1 Hash of 'kerala': 5d59a763175954d7fb31a5e31086fee2e1f4af5b
